Sailing The Seaborn! Below are the majority 4 topics here

1. Examine call volume and identify surge times, seasons, or events.

Let's try and do this by looking at peak hours and days. Then with that info we can look to see if there was a specific event that occured then? Seasonality or looking at sunrise/sunset times would be helpful as well.

2. Analyze trends in call content by looking for which issues are most frequent and what is the average count of issues per call.

This will likely be regex heavy? Would be searching for keywords within the call dialogue.

3. Assess regional and demographic trends in call content and call volume.

This is just like doing #2 but including demographics

4. Determine the count and success rate of imminent risk calls.

This seems the easiest, would be used best when paired with the other means of looking into the data

In [ ]:
pip install sunriset

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import sunriset
from datetime import datetime
from suntimes import SunTimes
from datetime import timedelta
import math
import pytz
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


%matplotlib inline

I am importing these items as it is fortold in the python commandments

In [ ]:
crisis1 = pd.read_excel('../data/Crisis1.xlsx', header=2)
crisis2 = pd.read_excel('../data/Crisis2.xlsx', header=2)

In [ ]:
crisis1.tail()

I'm converting the call times here to datetime format! Info & formatting pulled from here
https://www.geeksforgeeks.org/python-working-with-date-and-time-using-pandas/

In [ ]:
crisis1['CallDateAndTimeStart'] = pd.to_datetime(crisis1['CallDateAndTimeStart'])
crisis1['CallDateAndTimeEnd'] = pd.to_datetime(crisis1['CallDateAndTimeEnd'])
crisis1['CallDateStartWeekdate'] = crisis1['CallDateAndTimeStart'].dt.day_name()
#CallDateStartWeekdate give a Monday, Tuesday etc result
crisis1['CallDateStartTime'] = crisis1['CallDateAndTimeStart'].dt.hour
#CallDateStartTime gives the hour of the day the call was made, military time
crisis1['month'] = crisis1['CallDateAndTimeStart'].dt.month
#CallDateStartMonth gives the month which the call was made
crisis1['CallDateStartDay'] = crisis1['CallDateAndTimeStart'].dt.day
#CallDateStartDay gives us the DD
#crisis1['CallDateStartYYYYMMDD'] = crisis1['CallDateAndTimeStart'].dt.year + '-' + crisis1['CallDateAndTimeStart'].dt.month + '-' + crisis1['CallDateAndTimeStart'].dt.day
crisis1['CallDateStartDay']

Attempting to do a regex here so that I can make a kde seaborn plot for call length and call time of day, but I'm having an issue here of pulling the call time. I tried \d{2}:\d{2}:00 but that was coming up with nothing, somehow. I don't think colon is a strange character though? https://seaborn.pydata.org/generated/seaborn.kdeplot.html?highlight=kde%20plot#seaborn.kdeplot https://towardsdatascience.com/an-ultimate-cheatsheet-of-data-visualization-in-seaborn-be8ed13a3697 https://seaborn.pydata.org/tutorial/relational.html https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

In [ ]:
crisis1.dropna(how='all', axis=1, inplace=True)
crisis2.dropna(axis=1, how='all', inplace=True)

In [ ]:
stats_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=10, nrows=9, usecols='A:B')
stats_2020.rename(inplace=True, columns={'Total Calls':'Category', 'Unnamed: 1':'Total'})
#stats_2020['Total'].sub(' calls', '') this didn't work the way I wanted it to, commenting it out until I figure out how to fix it
stats_2020

In [ ]:
queue_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=23, nrows=12, usecols='A:I')
queue_dist_2020

In [ ]:
month_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=39, nrows=12, usecols='A:K')
month_dist_2020

In [ ]:
month_dist_2020['Month'].str.extract(r'((?<=\d{4}-)\d{2})')

In [ ]:
day_dist_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=55, nrows=7, usecols='A:K')
day_dist_2020

In [ ]:
queue_answered_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=66, nrows=10, usecols='A:C')
queue_answered_2020

In [ ]:
service_speed_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=80, nrows=10, usecols='A:D')
service_speed_2020

In [ ]:
abandon_rate_2020 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2020', skiprows=94, nrows=10, usecols='A:D')
abandon_rate_2020

In [ ]:
stats_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=10, nrows=9, usecols='A:B')
stats_2021.rename(inplace=True, columns={'Total Calls':'Category', 'Unnamed: 1':'Total'})
stats_2021

In [ ]:
queue_dist_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=23, nrows=11, usecols='A:I')
queue_dist_2021

In [ ]:
month_dist_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=38, nrows=12, usecols='A:K')
month_dist_2021

In [ ]:
hour_dist_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=54, nrows=24, usecols='A:K')
hour_dist_2021

In [ ]:
day_dist_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=82, nrows=7, usecols='A:K')
day_dist_2021

In [ ]:
service_speed_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=93, nrows=10, usecols='A:D')
service_speed_2021

In [ ]:
abandon_rate_2021 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2021', skiprows=107, nrows=10, usecols='A:D')
abandon_rate_2021

In [ ]:
stats_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=7, nrows=9, usecols='A:B')
stats_2022.rename(inplace=True, columns={'Total Calls':'Category', 'Unnamed: 1':'Total'})
stats_2022

In [ ]:
queue_dist_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=18, nrows=10, usecols='A:I')
queue_dist_2022

In [ ]:
month_dist_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=32, nrows=4, usecols='A:K')
month_dist_2022

In [ ]:
hour_dist_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=40, nrows=24, usecols='A:K')
hour_dist_2022

In [ ]:
day_dist_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=68, nrows=7, usecols='A:K')
day_dist_2022

In [ ]:
service_speed_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=80, nrows=10, usecols='A:D')
service_speed_2022

In [ ]:
abandon_rate_2022 = pd.read_excel('../data/call_volume.xlsx', sheet_name='2022', skiprows=94, nrows=10, usecols='A:D')
abandon_rate_2022

In [ ]:
#crisis1['CallDateAndTimeStart'] = crisis1['CallDateAndTimeStart'].astype('str')
#crisis1['CallDateStartTime'] = crisis1['CallDateAndTimeStart'].str.findall(r'\d{2}:\d{2}:00')
#crisis1['CallDateStartDate'] = crisis1['CallDateAndTimeStart'].str.findall(r'\d{4}-\d{2}-\d{2}')
#crisis1['CallDateStartDate']
#crisis1['CallDateStartTime']

DataFrame Cheat Sheet

2020:

stats_2020 -- high-level statistics about total number of calls, unanswered/disconnected rates, etc.

queue_dist_2020 -- distribution of calls across all queue services

month_dist_2020 -- distribution of calls by month

day_dist_2020 -- distribution of calls by day of week

queue_answered_2020 -- answered calls across all queue services

service_speed_2020 -- speed of answer in time categories

abandon_rate_2020 -- speed of caller abandoning call in time categories


2021:

stats_2021 -- high-level statistics about total number of calls, unanswered/disconnected rates, etc.

queue_dist_2021 -- distribution of calls across all queue services

month_dist_2021 -- distribution of calls by month

hour_dist_2021 -- distribution of calls by hour

day_dist_2021 -- distribution of calls by day of week

service_speed_2021 -- speed of answer in time categories

abandon_rate_2021 -- speed of caller abandoning call in time categories



2022:

stats_2022 -- high-level statistics about total number of calls, unanswered/disconnected rates, etc.

queue_dist_2022 -- distribution of calls across all queue services

month_dist_2022 -- distribution of calls by month

hour_dist_2022 -- distribution of calls by hour

day_dist_2022 -- distribution of calls by day of week

service_speed_2022 -- speed of answer in time categories

abandon_rate_2022 -- speed of caller abandoning call in time categories

color codes b: blue g: green r: red c: cyan m: magenta y: yellow k: black w: white

In [ ]:
crisis1.info()

In [ ]:
crisis1.describe(include='all')

In [ ]:
sns.barplot(data=crisis1, x="CallLength", y="CallDateStartWeekdate", 
            #hue="CRISIS Demographics - Gender Identity"
           )

Mondays am i right lol

g = sns.FacetGrid(
    crisis1, 
    row="CallDateStartMonth",
    #col="CRISIS Demographics - Gender Identity"
    )
g.map_dataframe(sns.histplot, x="CallDateStartTime")
#g.fig.set_size_inches(18, 10)
#g.set_axis_labels("Total Bill", "Frequency")

This wasn't working quite right and I need to relook at it before adding it into my thing

In [ ]:
sns.boxplot(data=crisis1, x="CallLength", y="CallDateStartWeekdate", showfliers = False)
#no outliers call time chart

In [ ]:
sns.boxplot(data=crisis1, x="CallDateStartTime", y="CallLength", showfliers = False)

Early morning (sunrise) has much lower call volume, while ~sunset~ tends to have more calls

In [ ]:
sns.histplot(data=crisis1, x="CallDateStartTime")

In [ ]:
sns.scatterplot(x="CallDateAndTimeStart", y="CallLength", data=crisis1)

nvm guess this heatmap doesn't quite work for what i'm attempting

In [ ]:
#sns.heatmap(crisis1['CallDateStartMonth'], crisis1['CallDateStartWeekdate'], crisis1['CallLength'], nnot=True, fmt="d")

In [ ]:
crisis1['CallDateStartWeekdate']

This isn't quite panning out how i expected it to.

sns.set_theme(style="dark")

# Plot each year's time series in its own facet
g = sns.relplot(
    data=crisis1,
    x="CallDateStartWeekdate", y="CallReportNum", col="CallDateStartMonth", hue="CallDateStartMonth",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.8, .85, year, transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=crisis1, x="CallDateStartWeekdate", y="CallReportNum", units="CallDateStartMonth",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])

# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("", "Call Volume")
g.tight_layout()

In [ ]:
import sunriset
import datetime
from datetime import timedelta
import math
import pytz

import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

I'm trying to add sunrise/sunset times to these charts, but first i really need to pull in the times of sunrise/sunset. Then maybe do a count of all the calls on that day, then work on charting items

In [ ]:
lat = 36.174465
long = -86.767960
local_tz = -5

number_of_years = 4
start_date = datetime.date(2019,1,1)

nashvilletime = sunriset.to_pandas(start_date, lat, long, local_tz, number_of_years)
nashvilletime.reset_index(inplace=True)
nashvilletime

In [ ]:
nashvilletime.plot(x='Solar Elevation Angle (degrees)', 
        y='Solar Azimuth Angle (degrees cw from North)', 
        color='orange',
        title='weird and strange loopy graph that somehow correlates to sunset time or something')
plt.show()

In [ ]:
nashvillesun = nashvilletime[['index','Sunrise','Sunset']]

In [ ]:
nashvillesun['Sunrise'] = nashvillesun['Sunrise'].astype('str')
nashvillesun['Sunset'] = nashvillesun['Sunset'].astype('str')
nashvillesun['Sunrise'] = nashvillesun['Sunrise'].str.replace('0 days ' , '')
nashvillesun['Sunset'] = nashvillesun['Sunset'].str.replace('0 days ' , '')
nashvillesun['Sunrise'] = nashvillesun['Sunrise'].str.replace(r'.\d{6}' , '')
nashvillesun['Sunset'] = nashvillesun['Sunset'].str.replace(r'.\d{6}' , '')
nashvillesun

In [ ]:
nashvillesun['index'] = nashvillesun['index'].astype('str')
nashvillesun['sunrisetime'] = (nashvillesun['index'] + ' ' + nashvillesun['Sunrise'])
nashvillesun['sunsettime'] = (nashvillesun['index'] + ' ' + nashvillesun['Sunset'])
nashvillesun['sunrisetime'] = pd.to_datetime(nashvillesun['sunrisetime'])
nashvillesun['sunsettime'] = pd.to_datetime(nashvillesun['sunsettime'])

nashvillesun

In [ ]:
sns.distplot((nashvilletime['Sunrise']))
sns.distplot((nashvilletime['Sunset']))

In [ ]:
plt.plot(nashvilletime[['Sunrise','Sunset']])
plt.title('Sunrise & Sunset')
plt.legend(nashvilletime[['Sunrise','Sunset']])
plt.grid(axis='y', alpha=0.5)
plt.show()

Ok, so I've done all this, I've imported in the sunrise sunset times into the fold, now i need to merge them onto the original dataset so all of this isn't for not.
I'm going to start by merging them on month, which i know is probably a bad move. I'll instead get an average of the month and work with a single number?

In [ ]:
nashvillesun['month'] = nashvillesun['sunrisetime'].dt.month

In [ ]:
crisis1['index'] = crisis1['CallDateAndTimeStart'].dt.date
crisis1['index'] = crisis1['index'].astype(str)
crisis1['index']

In what some are describing to be a miracle, I have added on the sunset and sunrise time onto a spreadsheet based on the day. I can't believe it.

In [ ]:
crisis00 = pd.merge(crisis1, nashvillesun, on = "index", how = "left")
crisis00 = crisis00[['CallDateAndTimeStart', 'CallDateAndTimeEnd', 'CallLength', 'index', 'Sunrise'
                    , 'Sunset', 'sunrisetime', 'sunsettime']]

In [ ]:
crisis00['Nighttime'] = (crisis00['CallDateAndTimeStart'].dt.time > crisis00['sunsettime'].dt.time) + (crisis00['CallDateAndTimeStart'].dt.time < crisis00['sunrisetime'].dt.time)

sns.countplot(y="Nighttime", data=crisis00, palette="viridis")

In [ ]:
sns.barplot(x=crisis00['CallDateAndTimeStart'].dt.hour, y=crisis00['CallLength'], hue=crisis00['Nighttime'], palette="viridis")

In [ ]:
copl = sns.countplot(x=crisis00['CallDateAndTimeStart'].dt.hour.astype('int'), hue=crisis00['Nighttime'], palette="viridis")

In [ ]:
sns.countplot(x=crisis00['CallDateAndTimeStart'].dt.day_name(), hue=crisis00['Nighttime'], palette="viridis")

In [ ]:
crisis00['month'] = crisis00['CallDateAndTimeStart'].dt.month
crisis00['hour'] = crisis00['CallDateAndTimeStart'].dt.hour
crisis00['year'] = crisis00['CallDateAndTimeStart'].dt.year
crisis00['day'] = crisis00['CallDateAndTimeStart'].dt.day
crisis00['weekday'] = crisis00['CallDateAndTimeStart'].dt.day_name()
crisis00.head()

In [ ]:
g = sns.catplot(x="month", hue="Nighttime",
                data=crisis00, kind="count",
                height=4, palette="viridis", 
                 aspect=1.2, legend_out=True,
               sharex=True, sharey=True).set(title='Count of Calls by Time of Day');

In [ ]:
g = sns.catplot(x="hour", hue="Nighttime", col="month",
                data=crisis00, kind="count",
                height=4, palette="viridis", 
                 aspect=1.3, legend_out=True,
               sharex=True, sharey=True);

In [ ]:
sns.countplot(x=crisis00['index'], hue=crisis00['Nighttime'], palette="viridis")

In [ ]:
g = sns.FacetGrid(data=crisis00, col="weekday", height=3.5, aspect=.65)
g.map(sns.histplot, "Nighttime")

In [ ]:
ax = sns.pointplot(x="month", y="CallLength", hue="Nighttime", data=crisis00, palette="viridis", ci=95, height=5, aspect=1).set(ylim=(0,20), title='Average Call Length')

In [ ]:
g = sns.catplot(x="weekday", y="CallLength", hue="Nighttime", data=crisis00, col="month", kind="point", palette="viridis").set(ylim=(0,20))